# Create pickle files from .pro files

#### Created by Megan Thompson-Munson (2023)

**Input:** 
- `/scratch/alpine/metm9666/project-2_output-4/*.pro`

**Output:** 
- `/scratch/alpine/metm9666/project-2_processed-output-4/*firn-data.p`
- `/scratch/alpine/metm9666/project-2_processed-output-4/*firn-data-interpolated.p`

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import xarray as xr
import datetime
import glob
from scipy import stats
from scipy.signal import savgol_filter
from scipy.interpolate import griddata
import pickle
import warnings
warnings.simplefilter('ignore', UserWarning)
warnings.filterwarnings('ignore')

In [2]:
# Dataset path
path_raw = '/scratch/alpine/metm9666/project-2_output-4/'
path_processed = '/scratch/alpine/metm9666/project-2_processed-output-4/pro-files_processed/'

## Functions for reading in and processing data

In [3]:
# Function to read proFile
def read_pro(proFile):

    # Read data line by line
    with open(proFile) as f:

        # Read in header information
        for h in range(39):
            header = f.readline()
            if h == 1:
                station = int(header[13:])
            if h == 2:
                lats = (float(header[10:-1]))
            if h == 3:
                lons = (float(header[11:-1]))

        # Create empty lists for each variable
        date = []
        depth = []
        thickness = []
        density = []
        temperature = []
        age = []
        water = []
        ice = []
        air = []

        # Read in data below header
        for line in f:
            if line.startswith('0500'): # Date
                date.append(datetime.datetime.strptime(line[5:24],'%d.%m.%Y %H:%M:%S'))
            if line.startswith('0501'): # Height
                heightArray = np.array(list(map(float,line.split(',')))[2:])/100
                depthArray = (heightArray-heightArray[-1])*-1
                depth.append(depthArray)
                thicknessShort = np.array(heightArray[1:]-heightArray[:-1])
                thicknessArray = np.insert(thicknessShort,0,heightArray[0])
                thickness.append(thicknessArray)
            if line.startswith('0502'): # Density
                density.append(np.array(list(map(float,line.split(',')))[2:]))
            if line.startswith('0503'): # Temperature
                temperature.append(np.array(list(map(float,line.split(',')))[2:]))         
            if line.startswith('0506'): # Ice
                water.append(np.array(list(map(float,line.split(',')))[2:])/100)  
            if line.startswith('0515'): # Water
                ice.append(np.array(list(map(float,line.split(',')))[2:])/100)                  
            if line.startswith('0516'): # Air
                air.append(np.array(list(map(float,line.split(',')))[2:])/100)

    # Close file
    f.close()

    fac = []
    for t in range(len(date)):
        fac.append(np.sum(thickness[t]*air[t]))
        
    data = {'date':date,'station':station,'lat':lats,'lon':lons,'fac':fac,
            'fac_smooth':savgol_filter(fac,501,2),
            'depth':depth,'thickness':thickness,'density':density,
            'temperature':temperature,'water':water,'ice':ice,'air':air}
    return data

In [4]:
# Function to interpolate onto regular vertical grid
def interpolate(pro,variable,maxdepth):
    
    t200 = np.linspace(0,200,len(pro['date']))
    
    if maxdepth<=10:
        z = np.arange(0,maxdepth+0.01,0.01)[::-1]
    else:
        z = np.concatenate([np.arange(0,10.01,0.01),np.arange(11,maxdepth+1,1)])[::-1]
        
    y_interp = []
    for i in range(len(t200)):
        zi = pro['depth'][i]
        yi = pro[variable][i][zi<=maxdepth]
        zi = zi[zi<=maxdepth]
        
        y_interp.append(griddata(zi,yi,z))
        
    return pd.DataFrame(data=np.stack(y_interp).T,index=z,columns=t200)

In [5]:
def split_timeseries(dec_pro,inc_pro):
    
    date_array = np.array(dec_pro['date'])
    keys = np.array(['date','fac','fac_smooth','depth',
                     'thickness','density','temperature','water','ice','air'])

    # Split data into initial (0-100), dec (100-200), and inc (100-200)
    initial = {}
    dec = {}
    inc = {}
    
    for i in range(len(keys)):

        # Convert to arrays
        dec_array = np.array(dec_pro[keys[i]])
        inc_array = np.array(inc_pro[keys[i]])

        # Add each array as new key to each dictionary
        initial[keys[i]] = dec_array[date_array<pd.to_datetime('2080-01-01 00:30:00')]
        dec[keys[i]] = dec_array[date_array>=pd.to_datetime('2080-01-01 00:30:00')]
        inc[keys[i]] = inc_array[date_array>=pd.to_datetime('2080-01-01 00:30:00')]
        
    # Add time as years
    initial['years'] = np.linspace(0,100,2610,endpoint='False')
    dec['years'] = np.linspace(100,200,2609)
    inc['years'] = np.linspace(100,200,2609)
    
    # Crate new dictionary to wrap all others and add in lat/lon
    data = {'station':dec_pro['station'],'lat':dec_pro['lat'],'lon':dec_pro['lon'],
            'initial':initial,'dec':dec,'inc':inc}
    return data

In [6]:
def apply_interpolation(dec_pro,inc_pro):
    
    # Interpolate density data
    rho_dec = interpolate(dec_pro,'density',100)
    rho_inc = interpolate(inc_pro,'density',100)

    # Zoom in on time/depths of interest (go slightly wider than needed)
    rho_dec_zoom = rho_dec.loc[31:0,89:111]
    rho_inc_zoom = rho_inc.loc[31:0,89:111]

    # Interpolate firn temperature data
    ft_dec = interpolate(dec_pro,'temperature',100)
    ft_inc = interpolate(inc_pro,'temperature',100)

    # Zoom in on time/depths of interest
    ft_dec_zoom = ft_dec.loc[31:0,89:111]
    ft_inc_zoom = ft_inc.loc[31:0,89:111]

    # Interpolate water content data
    water_dec = interpolate(dec_pro,'water',100)
    water_inc = interpolate(inc_pro,'water',100)

    # Zoom in on time/depths of interest
    water_dec_zoom = water_dec.loc[31:0,89:111]
    water_inc_zoom = water_inc.loc[31:0,89:111]
    
    return {'station':dec_pro['station'],'lat':dec_pro['lat'],'lon':dec_pro['lon'],
            'rho_dec':rho_dec_zoom,'rho_inc':rho_inc_zoom,
            'temp_dec':ft_dec_zoom,'temp_inc':ft_inc_zoom,
            'lwc_dec':water_dec_zoom,'lwc_inc':water_inc_zoom}

### Select data

In [7]:
# Cooling > warming (cooling eliminates melt): 636, 67

# Warming slightly > cooling (no melt): 1763, 1791, 1085

# Warming > cooling (warming creates melt): 549, 1542

# Warming > cooling (mean summer temp < 0, but melt >0): 1541, 619, 683

# Cooling > warming (warming depletes firn): 1026, 859

# Cooling > warming (Summer temp >0 and initial FAC ~0): 1718, 993

In [8]:
df = pd.read_csv('/scratch/alpine/metm9666/project-2_processed-output-4/FAC-change_vs_atm.csv')

In [9]:
# df[df['melt']>0][df['melt_cool']==0].sort_values(by=['melt'],ascending=False)
# df[df['melt']>0][df['melt_cool']==0][df['melt_warm']>=df['melt']].sort_values(by=['melt'],ascending=False).idx.values
# df[df['melt_warm']==0][df['summer_temp']<-10][df['warm_change']<=-2]
# df[df['melt']==0][df['melt_warm']>0].sort_values(by=['melt_warm'],ascending=False).iloc[0:10].idx.values
# df[df['summer_temp']>-6][df['summer_temp']<-4][df['cool_warm_diff']<0][df['melt']>0]
# df[df['summer_temp']>-3][df['summer_temp']<0][df['cool_warm_diff']>0][df['melt']>300].idx.values
df[df['melt']>700][df['summer_temp']>0].idx.values

array([ 993,  942,  372, 1718,  448, 1930,  881,  649,   81, 1199, 1737,
       1581, 1074, 1276, 1677, 1478, 1942, 1666,  739,  212,  595, 1600,
       1593, 1077, 1622, 1869, 1156, 1890, 1087, 1303,  678,  319, 1812,
       1766,  144,  846, 1344,  288,   52,  734,  442,  331,  717,  936,
        402,  675, 1702,  693,    6,  184,  472, 1565,  277, 1346, 1575,
       1157,  267,  156,  534,  633, 1698,  606, 1160,  892, 1283,  499,
       1351, 1903])

## Apply functions and export

In [10]:
# f = [1763,1542,683,859,232]
f = [993,  942,  372, 1718,  448, 1930,  881,  649,   81, 1199, 1737]
files = [[path_raw+'{}_TA_DEC_REPEAT1991.pro'.format(i),path_raw+'{}_TA_INC_REPEAT1991.pro'.format(i)] for i in f]

In [11]:
# Loop through, read, process, and export
for i in range(len(files)):
    pro_dec = read_pro(files[i][0])
    pro_inc = read_pro(files[i][1])

    data_ts = split_timeseries(pro_dec,pro_inc)
    pickle.dump(data_ts,
                open(path_processed+'{}_firn-data.p'.format(pro_dec['station']),'wb'))

    data_interp = apply_interpolation(pro_dec,pro_inc)
    pickle.dump(data_interp,
                open(path_processed+'{}_firn-data-interpolated.p'.format(pro_dec['station']),'wb'))

In [85]:
# # Gather all files and read in using function
# files = sorted(glob.glob(path_raw+'*.pro'))

# # Loop through, read, process, and export
# for i in np.arange(0,len(files),2):
    
#     if i!=136:
#         pro_dec = read_pro(files[i])
#         pro_inc = read_pro(files[i+1])

#         data_ts = split_timeseries(pro_dec,pro_inc)
#         pickle.dump(data_ts,
#                     open(path_processed+'{}_firn-data.p'.format(pro_dec['station']),'wb'))

#         data_interp = apply_interpolation(pro_dec,pro_inc)
#         pickle.dump(data_interp,
#                     open(path_processed+'{}_firn-data-interpolated.p'.format(pro_dec['station']),'wb'))